In [1]:
import zipfile

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
import math

### 相邻采样生成训练样本 

*
输入的X:
[[ 0.  1.  2.  3.  4.  5.],[18. 19. 20. 21. 22. 23.]]  
对应的Y(错开):
[[ 1.  2.  3.  4.  5.  6.],[19. 20. 21. 22. 23. 24.]]
*

In [2]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps):
    """Sample mini-batches in a consecutive order from sequential data."""
    corpus_indices = np.array(corpus_indices)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0 : batch_size * batch_len].reshape((
        batch_size, batch_len))
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i : i + num_steps]
        Y = indices[:, i + 1 : i + num_steps + 1]
        yield X, Y

### 文本处理类
- 文本预处理,包括减少频率少的词,过滤标点符号等
- 生成词到index 以及 index到词的字典
- 两个方法来对句子进行 text到index的arr 或者 index的arr到text的互转

In [14]:
import numpy as np
import string
from nltk.tokenize import word_tokenize
from collections import Counter

class TextConverter(object):
    def __init__(self, max_vocab=5000):
        """建立一个字符索引转换器
        Args:
            text_path: 文本位置
            max_vocab: 最大的单词数量
        """
        
        with zipfile.ZipFile('lyrics/jaychou_lyrics.txt.zip') as zin:
            with zin.open('jaychou_lyrics.txt') as f:
                text = f.read().decode('utf-8')
        text = text.replace('\n', ' ').replace('\r', ' ').replace('，', ' ').replace('。', ' ')
        text = self.preprocessing(text,max_vocab)

        
    def preprocessing(self,text,max_vocab):
        #tonization
        tokens = word_tokenize(text)
        
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]
        
        text = ' '.join(words)
        
        # 计算单词出现频率并排序
        vocab_count_list = Counter(text).most_common(max_vocab)
        
        # 如果超过最大值，截取频率最低的字符
        vocab = [x[0] for x in vocab_count_list]
        self.vocab = vocab

        self.word_to_int_table = {c: i for i, c in enumerate(self.vocab)}
        self.int_to_word_table = dict(enumerate(self.vocab))
        self.corpus_indices = self.text_to_arr(text)
        
        return text
        
    def vocab_size(self):
        return len(self.vocab)

    def word_to_int(self, word):
        if word in self.word_to_int_table:
            return self.word_to_int_table[word]
        else:
            raise Exception('Unknown word!')

    def int_to_word(self, index):
        if index == len(self.vocab):
            return '<unk>'
        elif index < len(self.vocab):
            return self.int_to_word_table[index]
        else:
            raise Exception('Unknown index!')

    def text_to_arr(self, text):
        arr = []
        for word in text:
            arr.append(self.word_to_int(word))
        return np.array(arr)

    def arr_to_text(self, arr):
        words = []
        for index in arr:
            words.append(self.int_to_word(index))
        return "".join(words)

In [15]:
convert = TextConverter(max_vocab=10000)

In [16]:
n_hidden = 512
n_embedding = 512
batch_size = 64
num_epochs = 20
vocab_size = convert.vocab_size()
num_steps = 20
top_n = 2 # 选取top几的词生成, 越大就随机性越大, 样本的多样性高
predict_len = 15 #生成多长的歌词

下面代码的问题：
我们输入的转化词向量之后是
X : [batch_size, n_step, vocab_size]

我们transpose了X
X : [n_step,batch_size,vocab_size]
这是代表，我把每个batch_size的第一行单独拧出来凑一起。
所以现在的情况是

假如batch_size是2 , n_step 是 5 ， 忽略最里面vocab_size
[
[[1,2,3,4,5]],
[[10,11,12,13,14]]
]

变成了

[
[1,10],[2,11],[3,12],[4,13],[5,14]
]

rnn 之后呢
outputs [n_step, batch_size, n_hidden]

经过view(-1,shape[2])
变成了
[
[1,10,2,11,3,12,4,13,5,14]
]
再经过dense层
[
[1,10,2,11,3,12,4,13,5,14]
]

这明显有问题

我们需要的是
[1,2,3,4,5,10,11,12,13,14]这样的

In [6]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        
        self.embed = nn.Embedding(vocab_size, n_hidden)
#         self.rnn = nn.RNN(input_size=vocab_size, hidden_size=n_hidden)
        self.rnn = nn.GRU(n_hidden, n_hidden, num_layers = 1)
        self.dense = nn.Linear(n_hidden, vocab_size)

    def forward(self, X, hidden):
        # embedding层的输入不需要one - hot
        X = self.embed(X)# X : [batch_size,n_step] -> # X : [batch_size,n_step, vocab_size]
        X = X.transpose(0, 1) # X : [n_step, batch_size, vocab_size]
        outputs, hidden = self.rnn(X, hidden)
        # outputs : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        le, mb, hd = outputs.shape
        outputs = outputs.view(le * mb, hd)
        outputs = self.dense(outputs) # [n_step * batch_size, num_classes]
        # outputs : each output of each time step
        # hidden : the hidden layer output at the last time step
        outputs = outputs.view(le, mb, -1)
        outputs = outputs.permute(1, 0, 2).contiguous()  # (batch, len, hidden)
        return outputs.view(-1, outputs.shape[2]), hidden

下面的代码为什么就可以呢

假如batch_size是2 , n_step 是 5 ， 忽略最里面vocab_size
[
[[1,2,3,4,5]],
[[10,11,12,13,14]]
]


转置后变成了

[
[1,10],[2,11],[3,12],[4,13],[5,14]
]

rnn后变成 (5,2,n_hidden)

view(10,n_hidden) 一下变成
[
[1,10,2,11,3,12,4,13,5,14]
]

dense后(10,vocab_size)
[
[1,10,2,11,3,12,4,13,5,14]
]

先回去原来的view(5,2,vocab_size)
[
[1,10],[2,11],[3,12],[4,13],[5,14]
]
在转置0,1 变成 (2,5,vocab_size)
[[1,2,3,4,5],[10,11,12,13,14]]
再continguous，重新分配内存
然后view成一列好进行loss的计算



In [7]:
class CharRNN(nn.Module):
    def __init__(self, num_classes, embed_dim, hidden_size, 
                 num_layers, dropout):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.word_to_vec = nn.Embedding(num_classes, embed_dim)
        self.rnn = nn.RNN(input_size=embed_dim, hidden_size=embed_dim)
        self.project = nn.Linear(hidden_size, num_classes)

    def forward(self, x, hs=None):
        batch = x.shape[0]
        if hs is None:
            hs = Variable(
                torch.zeros(self.num_layers, batch, self.hidden_size))
            if use_gpu:
                hs = hs.cuda()
        word_embed = self.word_to_vec(x)  # (batch, len, embed)
        word_embed = word_embed.permute(1, 0, 2)  # (len, batch, embed)
        out, h0 = self.rnn(word_embed, hs)  # (len, batch, hidden)
        le, mb, hd = out.shape
        out = out.view(le * mb, hd)
        out = self.project(out)
        out = out.view(le, mb, -1)
        out = out.permute(1, 0, 2).contiguous()  # (batch, len, hidden)
        return out.view(-1, out.shape[2]), h0

In [8]:
data_iter = data_iter_consecutive(
            convert.corpus_indices, batch_size = batch_size, num_steps = num_steps)

### pick_top_n是用来根据概率来采样生成的词
*如果top_n = 1, 默认生成概率最高的词, 这样的多样性不高*

In [9]:
def pick_top_n(preds, top_n=1):
    top_pred_prob, top_pred_label = torch.topk(preds, top_n, 1)
    top_pred_prob /= torch.sum(top_pred_prob)
    top_pred_prob = top_pred_prob.squeeze(0).detach().numpy()
    top_pred_label = top_pred_label.squeeze(0).detach().numpy()
    c = np.random.choice(top_pred_label, size=1, p=top_pred_prob)
    return c

### 预测生成句子函数

In [10]:
def predict_rnn(prefix, num_chars, model, vocab_size , idx_to_char,
                      char_to_idx, top_n = 2):
    # 初始化隐藏状态
    # hidden : [num_layers * num_directions, batch, hidden_size]
    hidden = torch.zeros(1, 1, n_hidden)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        #取output中最后一个词作为输入,求下一词的输入 + hidden output
        X = torch.LongTensor([[output[-1]]])
        (Y, hidden) = model(X, hidden)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1: #刚开始的提示词按照提示词加入
            output.append(char_to_idx[prefix[t + 1]])
        else:
            bestY = pick_top_n(Y, top_n = top_n)
            output.append(int(bestY))
    return ''.join([idx_to_char[i] for i in output])

In [11]:
model = TextRNN()
# model = CharRNN(vocab_size, 512, 512, 1, 0.5)

predict_rnn('分开', predict_len, model, convert.vocab_size , convert.int_to_word_table, convert.word_to_int_table,top_n)

'分开嘆朗确擀紧脾P卑仓部缭龟死正酸'

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 训练

In [13]:
for epoch in range(num_epochs):
    l_sum, n, start = 0.0, 0, time.time()
    data_iter = data_iter_consecutive(
            convert.corpus_indices, batch_size = batch_size, num_steps = num_steps)
    print('====epoch:'+ str(epoch+1) +"=====")
    for X, Y in data_iter:
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        hidden = torch.randn(1, batch_size, n_hidden)
        (output, hidden) = model(X, hidden)
        loss = criterion(output, Y.view(-1))    
        optimizer.zero_grad()
        loss.backward()   
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        l_sum += loss.item() * Y.shape[0]
        n += Y.shape[0]
    print('cost %.6f, perplexity %.2f, time %.2f sec' % (
                loss, math.exp(l_sum / n), time.time() - start))
    print(' -', predict_rnn(
        '珍惜', predict_len, model, convert.vocab_size, convert.int_to_word_table, convert.word_to_int_table,top_n))
    print(' -', predict_rnn(
        '天青色', predict_len, model, convert.vocab_size, convert.int_to_word_table, convert.word_to_int_table,top_n))
    print(' -', predict_rnn(
        '两个人', predict_len, model, convert.vocab_size, convert.int_to_word_table, convert.word_to_int_table,top_n))


====epoch:1=====
cost 5.701179, perplexity 440.96, time 11.78 sec
 - 珍惜 你的爱 你说你说你 你的手了
 - 天青色 我的爱 我不要我的 我不能 
 - 两个人 我不要 你的手了 我的爱你的
====epoch:2=====
cost 4.838658, perplexity 142.67, time 11.58 sec
 - 珍惜 你的爱 我的手 你的爱情 我
 - 天青色的 你的手 你的手中的 你说你
 - 两个人的手 我的爱情绪的 我的爱 你
====epoch:3=====
cost 4.151860, perplexity 69.02, time 11.65 sec
 - 珍惜 我们的脸上的 你说你的爱情 
 - 天青色的 你的手 你说你的手 你的手
 - 两个人的 我不要我的爱情绪 你说你的
====epoch:4=====
cost 3.624521, perplexity 39.02, time 11.57 sec
 - 珍惜 我不能再来 我的爱 你说不出
 - 天青色的 爱情悬崖的 我不要再见 我
 - 两个人 都是不会有 你说不了 我不能
====epoch:5=====
cost 3.161995, perplexity 24.32, time 11.63 sec
 - 珍惜 一统江山 这样的我 我不能够
 - 天青色的 时光 我想揍你的手 你说你
 - 两个人的 幸福呢 我不想再想 你的崩
====epoch:6=====
cost 2.741878, perplexity 16.16, time 11.45 sec
 - 珍惜一个 那么会有人 我的世界将你
 - 天青色 我们乘着阳 我不能再来 我想
 - 两个人的特 我的爱情悬在 我的爱 你
====epoch:7=====
cost 2.378531, perplexity 11.25, time 11.47 sec
 - 珍惜 我想揍你已经很难回头 我不想
 - 天青色 我的爱 我不能再来一个 我不
 - 两个人 都不公平也不能 不要再想你 
====epoch:8=====
cost 2.048591, perplexity 8.18, time 11.62 sec
 - 珍惜 我不能 别怪的 幸福中发芽 
 - 